In [13]:
####Only SBIR now
#### abstract can have "\n"
###

from bs4 import BeautifulSoup
import os
import pandas as pd
import csv
import json
import re

In [2]:
def filterHTMLstr(str):
    html_tag = {'&#xA;': '\n', '&quot;': '\"', '&amp;': '', '&lt;': '<', '&gt;': '>',
                '&apos;': "'", '&nbsp;': ' ', '&yen;': '¥', '&copy;': '©', '&divide;': '÷'
        , '&times;': 'x', '&trade;': '™', '&reg;': '®', '&sect;': '§', '&euro;': '€',
                '&pound;': '£', '&cent;': '￠', '&raquo;': '»', '&nbsp': ' ', u'\xa0': ' ',
                '\n': ' ', '\t': ' ', '    ': '', '&emsp': ' ',
                }
    for k, v in html_tag.items():
        str = str.replace(k, v)
        # str = str.replace(k[1:], v)
    str = str.strip('\n')
    str = str.strip(' ')

    return str

def to_CSV(totaldata):
    pd.DataFrame(totaldata).to_csv('./STTRResult/00_STTR_phase1.csv', index=False, encoding="utf-8")
    
def ReadFiles(Directory_path):  # Read all the html files
    path = Directory_path  # The directory
    file_names = os.listdir(path)  # 得到文件夹下的所有文件名称
    files_position = []
    for file_name in file_names:
        position = path + "/" + file_name
        # print(position)
        files_position.append(position)

    print("Total number of HTML files: ", len(files_position))
    return files_position


def MultipleFileProcess(files_position):
    totaldata = []
    for position in files_position:
        data = SingleHTMLProcess(position)
        totaldata.append(data)

    return totaldata

In [102]:
def SingleHTMLProcess(path):
    dic = {"Proposal Number": "", "Phase 1 Contract #": "", "Subtopic Title": "", "Proposal Title": "",
           "Small Business Concern_Firm": "", "Small Business Concern_Address": "", "Small Business Concern_Phone": "",
           "Principal Investigator_Name": "", "Principal Investigator_E-mail": "", "Principal Investigator_Address": "",
           "Principal Investigator_Phone": "",
           "Business Official_Name": "", "Business Official_E-mail": "", "Business Official_Address": "",
           "Business Official_Phone": "",
           "TRL_Begin": "", "TRL_End": "", "Technical Abstract": "", "Potential NASA applications": "",
           "Potential non-NASA applications": "","Technology Taxonomy Mapping":"", 'Research Topic':"", 'Research Institution Name':"", 'Research Institution Address':""}
    htmlfile = open(path, 'r', encoding='utf-8')
    html=htmlfile  
    bs = BeautifulSoup(html, "lxml")  # 缩进格式
    print(path)
    dic['Technical Abstract'] = filterHTMLstr(getTechAbstract(bs))
    dic['Potential NASA applications'] = filterHTMLstr(getNonNASAApplication(bs))
    dic['Proposal Number'] = filterHTMLstr(getProposalNumber(bs))
    dic['Research Topic'] = filterHTMLstr(getResearchTopic(bs))
    dic['Proposal Title'] = filterHTMLstr(getProjectTitle(bs))
    dic['Small Business Concern_Firm'] = filterHTMLstr(getSMBInfo(bs)[0])
    dic['Small Business Concern_Address'] = filterHTMLstr(getSMBInfo(bs)[1])
    dic['Principal Investigator_Name'] = filterHTMLstr(getInvestigatorInfo(bs))
    dic['Research Institution Name'] = filterHTMLstr(getResearchInstitutionInfo(bs)[0])
    dic['Research Institution Address'] = filterHTMLstr(getResearchInstitutionInfo(bs)[1])
    return dic

In [125]:
Directory_path = "../Datasets/00/sttr/phase1"
files_position = ReadFiles(Directory_path)
totaldata = MultipleFileProcess(files_position)
to_CSV(totaldata)

Total number of HTML files:  18
../Datasets/00/sttr/phase1/STTR-00-1-02-000021.html
../Datasets/00/sttr/phase1/STTR-00-1-01-000118.html
../Datasets/00/sttr/phase1/STTR-00-1-05-000079.html
../Datasets/00/sttr/phase1/STTR-00-1-02-000122.html
../Datasets/00/sttr/phase1/STTR-00-1-05-000025.html
../Datasets/00/sttr/phase1/STTR-00-1-01-000056.html
../Datasets/00/sttr/phase1/STTR-00-1-01-000104.html
../Datasets/00/sttr/phase1/STTR-00-1-03-000083.html
../Datasets/00/sttr/phase1/STTR-00-1-04-000022.html
../Datasets/00/sttr/phase1/STTR-00-1-05-000092.html
../Datasets/00/sttr/phase1/STTR-00-1-04-000062.html
../Datasets/00/sttr/phase1/STTR-00-1-04-000107.html
../Datasets/00/sttr/phase1/STTR-00-1-03-000070.html
../Datasets/00/sttr/phase1/STTR-00-1-03-000011.html
../Datasets/00/sttr/phase1/STTR-00-1-03-000142.html
../Datasets/00/sttr/phase1/STTR-00-1-04-000141.html
../Datasets/00/sttr/phase1/STTR-00-1-02-000063.html
../Datasets/00/sttr/phase1/STTR-00-1-03-000159.html


# Test

In [107]:
path = "../Datasets/00/sttr/phase1/STTR-00-1-01-000056.html"
htmlfile = open(path, 'r', encoding='utf-8')
html=htmlfile  
bs = BeautifulSoup(html, "lxml")  # 缩进格式

In [89]:
### Technical Abstract and Potential Commercial Application
def getTechAbstract(bs):
    technicalAbstract = bs.find_all("p")[1].get_text()
    return technicalAbstract

def getNonNASAApplication(bs):
    nonNASAApplication = bs.find_all("p")[3].get_text()
    return nonNASAApplication

In [90]:
### Proposal Number
def getProposalNumber(bs):
    proposalMine = bs.find_all("pre")[0].get_text()
    regProposalNum = re.compile('#: ([0-9]*)', re.S)
    proposalNumber = re.findall(regProposalNum, proposalMine)[0]
    return proposalNumber

In [91]:
### Research Topic and project title
def getResearchTopic(bs):
    topicAndTitleMine = bs.find_all("pre")[1].get_text()
    regResearchTopic = re.compile(r'Research Topic: (.*)\n', re.S)
    researchTopic = re.findall(regResearchTopic, topicAndTitleMine)[0]
    return researchTopic

def getProjectTitle(bs):
    topicAndTitleMine = bs.find_all("pre")[1].get_text()
    regProjectTitle = re.compile('Project Title: (.*)', re.S)
    projectTitle = re.findall(regProjectTitle, topicAndTitleMine)[0]
    return projectTitle

In [124]:
### Small Business and Investigator Info
def getSMBInfo(bs):
    smbAndInvestigator = bs.find_all("pre")[2].get_text()
    regSMBName = re.compile('Name:(.*?)Name', re.S)
    regSMBAddress1 = re.compile('Address:(.*?)City', re.S)
    regSMBAddress2 = re.compile('City:(.*?)ST', re.S)
    regSMBAddress3 = re.compile('ST:(.*?)Zip', re.S)
    regSMBAddress4 = re.compile('Zip: *([0-9]*)', re.S)
    # print("DEBUG: " + re.findall(regSMBAddress4, smbAndInvestigator)[0])
    smbName = re.findall(regSMBName, smbAndInvestigator)[0].strip()
    smbAddress1 = re.findall(regSMBAddress1, smbAndInvestigator)[0].strip()
    smbAddress2 = re.findall(regSMBAddress2, smbAndInvestigator)[0].strip()
    smbAddress3 = re.findall(regSMBAddress3, smbAndInvestigator)[0].strip()
    #smbAddress4 = re.findall(regSMBAddress4, smbAndInvestigator)[0].strip()
    smbAddress = smbAddress1 + ", " + smbAddress2 + ", " + smbAddress3 + ", " + smbAddress4
    return [smbName, smbAddress]
# print(getSMBInfo(bs))

In [104]:
def getInvestigatorInfo(bs):
    smbAndInvestigator = bs.find_all("pre")[2].get_text()
    regInvestigatorName = re.compile('Name:.*?Name:(.*?)Address', re.S)
    investigatorName = re.findall(regInvestigatorName, smbAndInvestigator)[0].strip()
    return investigatorName

In [93]:
### Research Institution Info
def getResearchInstitutionInfo(bs):
    researchInstitution = bs.find_all("pre")[3].get_text()
    regResearchInstitutionName = re.compile('Name:(.*?)Address', re.S)
    regResearchInstitutionAddress1 = re.compile('Address:(.*?)City', re.S)
    regResearchInstitutionAddress2 = re.compile('City:(.*?)ST', re.S)
    regResearchInstitutionAddress3 = re.compile('ST:(.*?)Zip', re.S)
    regResearchInstitutionAddress4 = re.compile('Zip: ([0-9]*)', re.S)
    researchInstitutionName = re.findall(regResearchInstitutionName, researchInstitution)[0].strip()
    researchInstitutionAddress1 = re.findall(regResearchInstitutionAddress1, researchInstitution)[0].strip()
    researchInstitutionAddress2 = re.findall(regResearchInstitutionAddress2, researchInstitution)[0].strip()
    researchInstitutionAddress3 = re.findall(regResearchInstitutionAddress3, researchInstitution)[0].strip()
    researchInstitutionAddress4 = re.findall(regResearchInstitutionAddress4, researchInstitution)[0].strip()
    researchInstitutionAddress = researchInstitutionAddress1 + ", " + researchInstitutionAddress2 + ", " + researchInstitutionAddress3 + ", " + researchInstitutionAddress4
    return [researchInstitutionName, researchInstitutionAddress]